In [ ]:
import apogee.tools.read as apread
import numpy as np
import matplotlib.pyplot as plt
import os

folder = '/Volumes/CoveyData-1/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'

def calcR(x, pos1=0, pos2=401):
    ccfCenter = 201
    pos1+= 1
    tempMirror = (x[ccfCenter:pos2])[::-1]
    sigmaA = np.sqrt(1.0 / (2.0 * len(tempMirror)) * np.sum((x[pos1:ccfCenter] - tempMirror)**2))
    return np.max(x) / (np.sqrt(2.0) * sigmaA)

def findInflection(x):
    maxPos = np.argmax(x)
    pos1, pos2 = -1, -1
    for i in range(maxPos, len(x) - 1):
        if (x[i + 1] - x[i] > 0):
            pos1 = i
            break
    for i in range(maxPos, 1, -1):
        if (x[i - 1] - x[i] > 0):
            pos2 = i
            break
    return pos2, pos1

def getMaxPositions(x, yBufferRange):
    temp = np.array(x)
    max1 = np.nanargmax(x)
    pos1, pos2 = findInflection(temp)
    temp[pos1:pos2] = np.nan
    temp[np.where(temp < 0.2)] = np.nan

    # If we can find one, record the second maximum
    try:
        max2 = np.nanargmax(temp)

        if (np.abs(max2 - pos1) < np.abs(max2 - pos2)):
            # Check if it's within the yBufferRange
            if (np.abs(x[max2] - x[pos1]) < yBufferRange):
                max2 = 'none'
        elif (np.abs(max2 - pos1) > np.abs(max2 - pos2)):
            if (np.abs(x[max2] - x[pos2]) < yBufferRange):
                max2 = 'none'
    except ValueError:
        max2 = 'none'
    
    # Double check that we are returning different positions
    if str(max1) == str(max2):
        max2 = 'none'
    
    return max1, max2

def recordTargets(targets, ranger, filename):
    targetCount = len(targets)

    if not os.path.exists(folder + str(round(ranger, 2)) + '/'):
        os.makedirs(folder + str(round(ranger, 2)) + '/')
    filename = folder + str(round(ranger, 2)) + '/' + filename + '.csv'
    f = open(filename, 'w')

    for i in range(targetCount):
        f.write("{0},{1},{2}\n".format(targets[i][0], targets[i][1], targets[i][2]))
    f.close()

def reportPositions(locationID, apogeeID, ranger, positions):
    if positions == []:
        return
    
    visitCount = len(positions)

    if not os.path.exists(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/'):
        os.makedirs(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/')
    filename = folder + str(round(ranger, 2)) + '/' + str(locationID) + '/' + str(apogeeID) + '.tbl'
    f = open(filename, 'w')

    
    f.write('visit\tmax1\tmax2\t')
    for i in range(len(positions[0][2])):
        f.write('r'+str(i+1)+'\t\t\t')
    f.write('\n')

    for i in range(visitCount):
        f.write(str(i + 1))
        position = positions[i]
        f.write('\t\t' + str(position[0]))
        f.write('\t\t' + str(position[1]))
        # record r values
        for val in position[2]:
            f.write('\t' + str(val))
        f.write('\n')
    f.close()

#------Calling SB9 SB2s
locationIDs = np.loadtxt('DR12_SB9_SB2s.lis', usecols=[5], unpack=True, dtype=float)
apogeeIDs = np.loadtxt('DR12_SB9_SB2s.lis', usecols=[4], unpack=True, dtype=str)
targetCount = len(apogeeIDs)

Candidates = []
skippedCand = []
outfile = open('SB9_Table.lis','w')
for i in range(targetCount):
    locationID = locationIDs[i]
    apogeeID = apogeeIDs[i]
    recorded = False
    for k in range(2, 50):
        ranger = k / 100.

        try:
            badheader, header = apread.apStar(locationID, apogeeID, ext=0, dr='13', header=True)
            data = apread.apStar(locationID, apogeeID, ext=9, header=False, dr='13')
        except IOError:
            skippedCand.append([locationID, apogeeID])
            break
        nvisits = header['NVISITS']

        positions = []
        for visit in range(1, nvisits):
            if (nvisits != 1):
                ccf = data['CCF'][0][1 + visit]
            else:
                ccf = data['CCF'][0]
            max1, max2 = getMaxPositions(ccf, ranger)

            r = []
            for cut in range(20):
                r.append(calcR(ccf, cut*10, (401 - (cut * 10))))
               
            if r < 8.0:
                if recorded is False:
                        recorded = True
                        Candidates.append([locationID, apogeeID, "r"])

            if str(max2) != 'none':
                if recorded is False:
                        recorded = True
                        Candidates.append([locationID, apogeeID, ranger])

            positions.append([max1, max2,r])

    outfile.write(str(locationID)+'\t'+ str(apogeeID)+'\t'+str(r)+'\n'+'\n')
outfile.close() 
print('Maximum R value',max(r))
print('Minimum R value',min(r))

#-----Calling Drew's List -------
locIDs = np.loadtxt('DrewsList_BC2.lis',usecols=[1],dtype=str)
apogIDs = np.loadtxt('DrewsList_BC2.lis',usecols=[0],dtype=str)
Count = len(apogIDs)

candidates = []
SkippedCand = []
out = open('DrewsList_Table.lis','w')
for i in range(Count):
    locID = locIDs[i]
    apogID = apogIDs[i]
    recorded = False
    for j in range(2, 50):
        Ranger = j / 100.

        try:
            badheader, header = apread.apStar(locID, apogID, ext=0, dr='13', header=True)
            Data = apread.apStar(locID, apogID, ext=9, header=False, dr='13')
        except IOError:
            SkippedCand.append([locID, apogID])
            break
        Nvisits = header['NVISITS']

        Positions = []
        for Visit in range(1, Nvisits):
            if (Nvisits != 1):
                CCF = Data['CCF'][0][1 + Visit]
            else:
                CCF = Data['CCF'][0]
            max1, max2 = getMaxPositions(CCF, Ranger)

            R = []
            for cuts in range(20):
                R.append(calcR(CCF, cuts*10, (401 - (cuts * 10))))
               
            if R < 8.0:
                if recorded is False:
                        recorded = True
                        candidates.append([locID, apogID, "r"])

            if str(max2) != 'none':
                if recorded is False:
                        recorded = True
                        candidates.append([locID, apogID, Ranger])

            Positions.append([max1, max2,R])

    out.write(str(locID)+'\t'+ str(apogID)+'\t'+str(R)+'\n'+'\n')
out.close() 
print('Maximum R value',max(R))
print('Minimum R value',min(R))

#----Random 2,000 from DR13
apog_randIDs = np.loadtxt('master_apStarList_DR13.csv',skiprows=1,usecols=[3],dtype=str)
loc_randIDs = np.loadtxt('master_apStarList_DR13.csv',skiprows=1,usecols=[4],dtype=str)
Total = len(apog_randIDs[1000:3000])
cands = []
SkippedCands = []
anotherfile = open('RandomSample_Table.lis','w')
for i in range(Total):
    loc_randID = loc_randIDs[i]
    apog_randID = apog_randIDs[i]
    recorded = False
    for k in range(2, 50):
        Ranger_value = k / 100.

        try:
            badheader, header = apread.apStar(loc_randID, apog_randID, ext=0, dr='13', header=True)
            z = apread.apStar(loc_randID, apog_randID, ext=9, header=False, dr='13')
        except IOError:
            SkippedCands.append([loc_randID, apog_randID])
            break
        N_visits = header['NVISITS']

        Pos = []
        for Visits in range(1, N_visits):
            if (Nvisits != 1):
                Ccf = z['CCF'][0][1 + Visits]
            else:
                Ccf = z['CCF'][0]
            max1, max2 = getMaxPositions(Ccf, Ranger_value)

            Rs = []
            for Cut in range(20):
                Rs.append(calcR(Ccf, Cut*10, (401 - (Cut * 10))))
               
            if Rs < 8.0:
                if recorded is False:
                        recorded = True
                        cands.append([loc_randID, apog_randID, "r"])

            if str(max2) != 'none':
                if recorded is False:
                        recorded = True
                        cands.append([loc_randID, apog_randID, Ranger_value])

            Pos.append(Rs)

    anotherfile.write(str(loc_randID)+'\t'+ str(apog_randID)+'\t'+str(Rs)+'\n'+'\n')
anotherfile.close() 
print("done")

/Users/reynaa/anaconda/envs/yo/lib/python2.7/site-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in less


('Maximum R value', 36.058144214157217)
('Minimum R value', 22.406753845683212)


In [ ]:
SB9_R = np.loadtxt('SB9_Table.lis',usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],dtype=str)
total = len(SB9_R)

R1_SB9 = []
R2_SB9 = []
R3_SB9 = []
R4_SB9 = []
R5_SB9 = []
R6_SB9 = []
R7_SB9 = []
R8_SB9 = []
R9_SB9 = []
R10_SB9 = []
R11_SB9 = []
R12_SB9 = []
R13_SB9 = []
R14_SB9 = []
R15_SB9 = []
R16_SB9 = []
R17_SB9 = []
R18_SB9 = []
R19_SB9 = []

for i in range(total):
    R1_SB9.append(SB9_R[i][0]) 
    R2_SB9.append(SB9_R[i][1])
    R3_SB9.append(SB9_R[i][2])
    R4_SB9.append(SB9_R[i][3])
    R5_SB9.append(SB9_R[i][4])
    R6_SB9.append(SB9_R[i][5])
    R7_SB9.append(SB9_R[i][6])
    R8_SB9.append(SB9_R[i][7])
    R9_SB9.append(SB9_R[i][8])
    R10_SB9.append(SB9_R[i][9])
    R11_SB9.append(SB9_R[i][10])
    R12_SB9.append(SB9_R[i][11])
    R13_SB9.append(SB9_R[i][12])
    R14_SB9.append(SB9_R[i][13])
    R15_SB9.append(SB9_R[i][14])
    R16_SB9.append(SB9_R[i][15])
    R17_SB9.append(SB9_R[i][16])
    R18_SB9.append(SB9_R[i][17])
    
R1_SB9 = np.array([s.strip('[,') for s in R1_SB9]).astype(float)
R2_SB9 = np.array([s.strip('[,') for s in R2_SB9]).astype(float)
R3_SB9 = np.array([s.strip('[,') for s in R3_SB9]).astype(float)
R4_SB9 = np.array([s.strip('[,') for s in R4_SB9]).astype(float)
R5_SB9 = np.array([s.strip('[,') for s in R5_SB9]).astype(float)
R6_SB9 = np.array([s.strip('[,') for s in R6_SB9]).astype(float)
R7_SB9 = np.array([s.strip('[,') for s in R7_SB9]).astype(float)
R8_SB9 = np.array([s.strip('[,') for s in R8_SB9]).astype(float)
R9_SB9 = np.array([s.strip('[,') for s in R9_SB9]).astype(float)
R10_SB9 = np.array([s.strip('[,') for s in R10_SB9]).astype(float)
R11_SB9 = np.array([s.strip('[,') for s in R11_SB9]).astype(float)
R12_SB9 = np.array([s.strip('[,') for s in R12_SB9]).astype(float)
R13_SB9 = np.array([s.strip('[,') for s in R13_SB9]).astype(float)
R14_SB9 = np.array([s.strip('[,') for s in R14_SB9]).astype(float)
R15_SB9 = np.array([s.strip('[,') for s in R15_SB9]).astype(float)
R16_SB9 = np.array([s.strip('[,') for s in R16_SB9]).astype(float)
R17_SB9 = np.array([s.strip('[,') for s in R17_SB9]).astype(float)
R18_SB9 = np.array([s.strip('[,') for s in R18_SB9]).astype(float)



DL_R = np.loadtxt('DrewsList_Table.lis',usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],dtype=str)
amount = len(DL_R)

R1_DL = []
R2_DL = []
R3_DL = []
R4_DL = []
R5_DL = []
R6_DL = []
R7_DL = []
R8_DL = []
R9_DL = []
R10_DL = []
R11_DL = []
R12_DL = []
R13_DL = []
R14_DL = []
R15_DL = []
R16_DL = []
R17_DL = []
R18_DL = []
R19_DL= []

for i in range(amount):
    R1_DL.append(DL_R[i][0]) 
    R2_DL.append(DL_R[i][1])
    R3_DL.append(DL_R[i][2])
    R4_DL.append(DL_R[i][3])
    R5_DL.append(DL_R[i][4])
    R6_DL.append(DL_R[i][5])
    R7_DL.append(DL_R[i][6])
    R8_DL.append(DL_R[i][7])
    R9_DL.append(DL_R[i][8])
    R10_DL.append(DL_R[i][9])
    R11_DL.append(DL_R[i][10])
    R12_DL.append(DL_R[i][11])
    R13_DL.append(DL_R[i][12])
    R14_DL.append(DL_R[i][13])
    R15_DL.append(DL_R[i][14])
    R16_DL.append(DL_R[i][15])
    R17_DL.append(DL_R[i][16])
    R18_DL.append(DL_R[i][17])
    
R1_DL = np.array([s.strip('[,') for s in R1_DL]).astype(float)
R2_DL = np.array([s.strip('[,') for s in R2_DL]).astype(float)
R3_DL = np.array([s.strip('[,') for s in R3_DL]).astype(float)
R4_DL = np.array([s.strip('[,') for s in R4_DL]).astype(float)
R5_DL = np.array([s.strip('[,') for s in R5_DL]).astype(float)
R6_DL = np.array([s.strip('[,') for s in R6_DL]).astype(float)
R7_DL = np.array([s.strip('[,') for s in R7_DL]).astype(float)
R8_DL = np.array([s.strip('[,') for s in R8_DL]).astype(float)
R9_DL = np.array([s.strip('[,') for s in R9_DL]).astype(float)
R10_DL = np.array([s.strip('[,') for s in R10_DL]).astype(float)
R11_DL = np.array([s.strip('[,') for s in R11_DL]).astype(float)
R12_DL = np.array([s.strip('[,') for s in R12_DL]).astype(float)
R13_DL = np.array([s.strip('[,') for s in R13_DL]).astype(float)
R14_DL = np.array([s.strip('[,') for s in R14_DL]).astype(float)
R15_DL = np.array([s.strip('[,') for s in R15_DL]).astype(float)
R16_DL = np.array([s.strip('[,') for s in R16_DL]).astype(float)
R17_DL = np.array([s.strip('[,') for s in R17_DL]).astype(float)
R18_DL = np.array([s.strip('[,') for s in R18_DL]).astype(float)

print('Max for R2 window for SB9',max(R2_SB9))
print('Max for R2 window for Drews List', max(R2_DL))
print('Min for R2 window for SB9',min(R2_SB9))
print('Min for R2 window for Drews List', min(R2_DL))


#Plot the histograms
np.histogram(R1_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R1_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R1_DL, bins=40,alpha=0.5,label='Drews List')
plt.hist(R1_SB9, bins=40,alpha=0.5,label='SB9_SB2s')
plt.title("Histogram of Confirmed Binaries")
plt.xlabel('R Values')
plt.text(40,70,'Histogram for the first R-window ($R_1$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL_R1_Histogram.png')

#-----
# Try to find standard deviation, mean, and bounds of histogram.
from scipy import stats
import numpy as np
import matplotlib.pylab as plt

np.histogram(R1_DL,range = None, normed = True, weights = None, density = None)
plt.hist(R1_DL, bins='auto',alpha=0.5,label='Drews List')
# Find the minimum and maximum of xticks (so we know where to compute a distribution)
xt = plt.xticks()[0]
xmin,xmax = min(xt),max(xt)
lnspc = np.linspace(xmin,xmax,len(R1_DL))

#Normal distribution
stdev = stats.norm.fit(R1_DL)
theo_int = stats.norm.pdf(lnspc,stdev)
plt.plot(lnspc,theo_int,label='Normal Distribution')

#Gamma Function Distribution
ag,bg,cg = stats.gamma.fit(R1_DL)
pdf_gamma = stats.gamma.pdf(lnspc,ag,bg,cg)
plt.plot(lnspc,pdf_gamma,label='Gamma Distribution')

#Beta Fuction Distribution
ab,bb,cb,db, = stats.beta.fit(R1_DL)
pdf_beta = stats.beta.pdf(lnspc,ab,bb,cb,db)
plt.plot(lnspc,pdf_beta,label='Beta Distribution')



In [4]:
#For the overlap of all three (SB9 SB2s, Drew's List, and First 20,000 from DR13)
rando = np.loadtxt('RandomSample_Table.lis',usecols=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],dtype=str)
Sum = len(rando)

R1_DR13 = []
R2_DR13 = []
R3_DR13 = []
R4_DR13 = []
R5_DR13 = []
R6_DR13 = []
R7_DR13 = []
R8_DR13 = []
R9_DR13 = []
R10_DR13 = []
R11_DR13 = []
R12_DR13 = []
R13_DR13 = []
R14_DR13 = []
R15_DR13 = []
R16_DR13 = []
R17_DR13 = []
R18_DR13 = []
R19_DR13= []

for i in range(Sum):
    R1_DR13.append(rando[i][0]) 
    R2_DR13.append(rando[i][1])
    R3_DR13.append(rando[i][2])
    R4_DR13.append(rando[i][3])
    R5_DR13.append(rando[i][4])
    R6_DR13.append(rando[i][5])
    R7_DR13.append(rando[i][6])
    R8_DR13.append(rando[i][7])
    R9_DR13.append(rando[i][8])
    R10_DR13.append(rando[i][9])
    R11_DR13.append(rando[i][10])
    R12_DR13.append(rando[i][11])
    R13_DR13.append(rando[i][12])
    R14_DR13.append(rando[i][13])
    R15_DR13.append(rando[i][14])
    R16_DR13.append(rando[i][15])
    R17_DR13.append(rando[i][16])
    R18_DR13.append(rando[i][17])
    
R1_DR13 = np.array([s.strip('[,') for s in R1_DR13]).astype(float)
R2_DR13 = np.array([s.strip('[,') for s in R2_DR13]).astype(float)
R3_DR13 = np.array([s.strip('[,') for s in R3_DR13]).astype(float)
R4_DR13 = np.array([s.strip('[,') for s in R4_DR13]).astype(float)
R5_DR13 = np.array([s.strip('[,') for s in R5_DR13]).astype(float)
R6_DR13 = np.array([s.strip('[,') for s in R6_DR13]).astype(float)
R7_DR13 = np.array([s.strip('[,') for s in R7_DR13]).astype(float)
R8_DR13 = np.array([s.strip('[,') for s in R8_DR13]).astype(float)
R9_DR13 = np.array([s.strip('[,') for s in R9_DR13]).astype(float)
R10_DR13 = np.array([s.strip('[,') for s in R10_DR13]).astype(float)
R11_DR13 = np.array([s.strip('[,') for s in R11_DR13]).astype(float)
R12_DR13 = np.array([s.strip('[,') for s in R12_DR13]).astype(float)
R13_DR13 = np.array([s.strip('[,') for s in R13_DR13]).astype(float)
R14_DR13 = np.array([s.strip('[,') for s in R14_DR13]).astype(float)
R15_DR13 = np.array([s.strip('[,') for s in R15_DR13]).astype(float)
R16_DR13 = np.array([s.strip('[,') for s in R16_DR13]).astype(float)
R17_DR13 = np.array([s.strip('[,') for s in R17_DR13]).astype(float)
R18_DR13 = np.array([s.strip('[,') for s in R18_DR13]).astype(float)


#for second R window
np.histogram(R2_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R2_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R2_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R2_DR13, bins='auto',alpha=0.5,label='1,000-3,000 of DR13')
plt.hist(R2_DL, bins='auto',alpha=0.5,label='Drews List')
plt.hist(R2_SB9, bins='auto',alpha=0.9,label='SB9_SB2s')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(140,700,'Histogram for the second R-window ($R_{2}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13(New)_R2_Histogram.png')
plt.show()

IndexError: list index out of range

In [3]:
#For 3rd R window
np.histogram(R2_DR13,range = None, normed = 1, weights = None, density = None)
np.histogram(R2_DL,range = None, normed = 1, weights = None, density = None)
np.histogram(R2_SB9,range = None, normed = 1, weights = None, density = None)
plt.hist(R2_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R2_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R2_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 3rd R-window ($R_3$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R3_Histogram.png')
plt.show()

NameError: name 'R2_DR13' is not defined

In [160]:
#For 4th R window
np.histogram(R3_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R3_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R3_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R3_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R3_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R3_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 4th R-window ($R_4$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R4_Histogram.png')
plt.show()

In [161]:
#For 5th R window
np.histogram(R4_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R4_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R4_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R4_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R4_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R4_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 5th R-window ($R_5$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R5_Histogram.png')
plt.show()

In [162]:
#For 6th R window
np.histogram(R5_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R5_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R5_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R5_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R5_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R5_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 6th R-window ($R_6$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R6_Histogram.png')
plt.show()

In [163]:
#For 7th R window
np.histogram(R6_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R6_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R6_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R6_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R6_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R6_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 7th R-window ($R_7$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R7_Histogram.png')
plt.show()

In [164]:
#For 8th R window
np.histogram(R7_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R7_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R7_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R7_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R7_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R7_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 8th R-window ($R_8$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R8_Histogram.png')
plt.show()

In [165]:
#For the 9th R Window
np.histogram(R8_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R8_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R8_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R8_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R8_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R8_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 9th R-window ($R_{9}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R9_Histogram.png')
plt.show()

In [166]:
# For 10th R Window
np.histogram(R9_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R9_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R9_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R9_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R9_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R9_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 10th R-window ($R_{10}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R10_Histogram.png')
plt.show()

In [167]:
#For 11th R Window
np.histogram(R10_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R10_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R10_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R10_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R10_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R10_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 11th R-window ($R_{11}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R11_Histogram.png')
plt.show()

In [168]:
#For 12th R window
np.histogram(R11_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R11_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R11_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R11_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R11_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R11_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 12th R-window ($R_{12}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R12_Histogram.png')
plt.show()

In [169]:
#For 13th R window
np.histogram(R12_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R12_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R12_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R12_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R12_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R12_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,400,'Histogram for the 13th R-window ($R_{13}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R13_Histogram.png')
plt.show()

In [170]:
#For the 14th R Window
np.histogram(R13_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R13_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R13_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R13_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R13_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R13_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 14th R-window ($R_{14}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R14_Histogram.png')
plt.show()

In [171]:
#For the 15th R window
np.histogram(R14_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R14_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R14_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R14_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R14_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R14_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 15th R-window ($R_{15}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R15_Histogram.png')
plt.show()

In [172]:
#For the 16th R window
np.histogram(R15_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R15_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R15_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R15_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R15_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R15_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 16th R-window ($R_{16}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R16_Histogram.png')
plt.show()

In [173]:
#For the 17th R Window
np.histogram(R16_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R16_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R16_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R16_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R16_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R16_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 17th R-window ($R_{17}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R17_Histogram.png')
plt.show()

In [177]:
#For 18th R window
np.histogram(R17_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R17_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R17_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R17_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R17_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R17_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 18th R-window ($R_{18}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R18_Histogram.png')
plt.show()

#For 18th R window
np.histogram(R17_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R17_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R17_DL, bins='auto',alpha=0.5,label='Drews List')
plt.hist(R17_SB9, bins='auto',alpha=0.9,label='SB9_SB2s')
plt.title("Histogram of Confirmed Binaries")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 18th R-window ($R_{18}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL_R18_Histogram.png')
plt.show()

In [175]:
#For 19th R window
np.histogram(R18_DR13,range = None, normed = True, weights = None, density = None)
np.histogram(R18_DL,range = None, normed = True, weights = None, density = None)
np.histogram(R18_SB9,range = None, normed = True, weights = None, density = None)
plt.hist(R18_DR13, bins='auto',alpha=0.5,label='First 20,000 of DR13')
plt.hist(R18_DL, bins='auto',alpha=0.5,label='Drews List',color='yellow')
plt.hist(R18_SB9, bins='auto',alpha=0.9,label='SB9_SB2s',color='red')
plt.title("Histogram of Confirmed Binaries and DR13 Sample")
plt.xlabel('R Values')
plt.text(100,500,'Histogram for the 19th R-window ($R_{19}$)',fontsize=10)
plt.legend(loc = 'upper right')
plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/hist/SB9.DL.DR13_R19_Histogram.png')
plt.show()